# Objective
To implement the techniques learnt as a part of the course.

## Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path="/content/drive/My Drive/AIML/Project 8 - Natural Language Processing/"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install contractions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re, string, unicodedata
import contractions                                     
from bs4 import BeautifulSoup                           
import nltk  

nltk.download('punkt')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

     |████████████████████████████████| 317kB 6.9MB/s 
     |████████████████████████████████| 245kB 27.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81699 sha256=6a8f4da95ec8fc30e2c606d9881f9c00d050358543a779f6c18badd1dd51912f
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## Import Dataset

In [ ]:
# Read the file
df_orig=pd.read_csv(data_path+'Tweets.csv')

## EDA

In [ ]:
df_orig.info()       # Info about dataframe, columns, data types, Null values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [ ]:
# Print the shape of dataset
df_orig.shape 

(14640, 15)

In [ ]:
# Print first five rows of dataset
df_orig.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
print('-----Number of Nulls-----')
df_orig.isnull().sum()

-----Number of Nulls-----


tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [ ]:
print('-----Number of unique values-----')
df_orig.nunique()

-----Number of unique values-----


tweet_id                        14485
airline_sentiment                   3
airline_sentiment_confidence     1023
negativereason                     10
negativereason_confidence        1410
airline                             6
airline_sentiment_gold              3
name                             7701
negativereason_gold                13
retweet_count                      18
text                            14427
tweet_coord                       832
tweet_created                   14247
tweet_location                   3081
user_timezone                      85
dtype: int64

In [ ]:
df_orig.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


* As we from above there are 15 columns. 
* Some of the columns do have null values.
* The shape of data frame is 14640 rows and 15 columns
* Also, there are no null values for 'airline_sentiment' and 'text' column which we will use

## Create a new dataset with columns 'text' and 'airline_sentiment' only

In [ ]:
df = df_orig[['text', 'airline_sentiment']]

In [ ]:
# Print the shape of new dataset
df.shape

(14640, 2)

In [ ]:
# Print first five rows of new dataset
df.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [ ]:
# Check the value count of airline_sentiment
df.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [ ]:
# Encode target categorical variable (airline_sentiment)
le = preprocessing.LabelEncoder()
df.airline_sentiment = le.fit_transform(df.airline_sentiment)
df.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,1
1,@VirginAmerica plus you've added commercials t...,2
2,@VirginAmerica I didn't today... Must mean I n...,1
3,@VirginAmerica it's really aggressive to blast...,0
4,@VirginAmerica and it's a really big bad thing...,0


In [ ]:
df.airline_sentiment.value_counts()

0    9178
1    3099
2    2363
Name: airline_sentiment, dtype: int64

--> "0" represents "negative" sentiments

--> "1" represents "neutral" sentiments

--> "2" represents "positive" sentiments

## Text Pre-processing and Data Preparation

In [ ]:
## Define functions for text pre-processing

## Remove HTML tags
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

## Remove contractions
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

## Remove the numbers
def remove_numbers(words):
  new_words = []                        # Create empty list to store pre-processed words.
  for word in words:
    new_word = re.sub(r'\d+', '', word)
    if new_word != '':
       new_words.append(new_word)    # Append processed words to new list.
  return new_words

## Remove special characters
def remove_special_characters(words):
  new_words = []                        # Create empty list to store pre-processed words.
  for word in words:
     new_word = re.sub('\W+',' ', word)
     if new_word != '':
        new_words.append(new_word)    # Append processed words to new list.
  return new_words

## Remove punctuations
def remove_punctuation(words):
    new_words = []                        # Create empty list to store pre-processed words.
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)    # Append processed words to new list.
    return new_words

## Convert to lower case
def to_lowercase(words):
    new_words = []                        # Create empty list to store pre-processed words.
    for word in words:
        new_word = word.lower()           
        new_words.append(new_word)        # Append processed words to new list.
    return new_words

## Lemmatize
def lemmatize_list(words):
    lemmatizer = WordNetLemmatizer()
    new_words = []
    for word in words:
      new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words

## Normalize function
def normalize(words):
    words = remove_numbers(words)
    words = remove_special_characters(words)
    words = remove_punctuation(words)
    words = to_lowercase(words)
    words = lemmatize_list(words)
    return ' '.join(words)    

In [ ]:
## Apply Text preprocessing
df['text'] = df['text'].apply(lambda x: strip_html(x))              # Remove HTML tags
df['text'] = df['text'].apply(lambda x: replace_contractions(x))    # Replace contractions (e.g. I've, You've etc.)
df.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,1
1,@VirginAmerica plus you have added commercials...,2
2,@VirginAmerica I did not today... Must mean I ...,1
3,@VirginAmerica it is really aggressive to blas...,0
4,@VirginAmerica and it is a really big bad thin...,0


In [ ]:
## Tokenization 
df['text'] = df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
df.head()

,text,airline_sentiment
0,"[@, VirginAmerica, What, @, dhepburn, said, .]",1
1,"[@, VirginAmerica, plus, you, have, added, com...",2
2,"[@, VirginAmerica, I, did, not, today, ..., Mu...",1
3,"[@, VirginAmerica, it, is, really, aggressive,...",0
4,"[@, VirginAmerica, and, it, is, a, really, big...",0


In [ ]:
## Normalize the data and join the words back to the text string
df['text'] = df.apply(lambda row: normalize(row['text']), axis=1)
df.head()

,text,airline_sentiment
0,virginamerica what dhepburn say,1
1,virginamerica plus you have add commercials ...,2
2,virginamerica i do not today must mean i n...,1
3,virginamerica it be really aggressive to bla...,0
4,virginamerica and it be a really big bad thi...,0


## Vectorization

In [ ]:
## Count vectorizer
vectorizer = CountVectorizer(max_features=1000)                # Keep only 1000 features as number of features will increase the processing time.
data_count = vectorizer.fit_transform(df['text'])

data_count = data_count.toarray()                        # Convert the data features to array.
data_count.shape

(14640, 1000)

In [ ]:
## TFIDF vectorizer
vectorizer = TfidfVectorizer(max_features=1000)
data_tfidf = vectorizer.fit_transform(df['text'])

data_tfidf = data_tfidf.toarray()                        # Convert the data features to array.
data_tfidf.shape

(14640, 1000)

## Use Random Forest model

In [ ]:
# Get target column
labels = df['airline_sentiment']

# Split data into training and testing set for Count Vector
x_train, x_test, y_train, y_test = train_test_split(data_count, labels, test_size=0.3, random_state=42)

In [ ]:
forest = RandomForestClassifier(n_estimators=10, n_jobs=4)
forest = forest.fit(x_train, y_train)
print(forest)

print(np.mean(cross_val_score(forest, data_count, labels, cv=10)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
0.7165983606557378


In [ ]:
# Predict the result
result = forest.predict(x_test)

# Print confusion matrix
conf_mat = confusion_matrix(y_test, result)
print(conf_mat)

[[2622  146   46]
 [ 486  351   47]
 [ 253  103  338]]


In [ ]:
# Split data into training and testing set for TFIDF Vector
x_train, x_test, y_train, y_test = train_test_split(data_tfidf, labels, test_size=0.3, random_state=42)

In [ ]:
forest = RandomForestClassifier(n_estimators=10, n_jobs=4)
forest = forest.fit(x_train, y_train)
print(forest)

print(np.mean(cross_val_score(forest, data_count, labels, cv=10)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
0.7204918032786886


In [ ]:
# Predict the result
result = forest.predict(x_test)

# Print confusion matrix
conf_mat = confusion_matrix(y_test, result)
print(conf_mat)

[[2653  123   38]
 [ 478  354   52]
 [ 244  106  344]]


## Summary

* We used the text of the tweet and the sentiment represented by the tweet which could be either positive, negative or neutral.
* The objective was to build a classification model.
* The text pre-processing was done by removing HTML tags, replacing contractions, removing numbers, special characters and punctuations. We also converted text to lower case and used lemmatization for tokenized words.
* This pre-processed data was then converted to numbers using vectorization techniques; count vectorization and TFIDF vectorization to be used for random forest classifier.
* The random forest classifier was used for predicting the results.
* For count vectorization technique, we got a cross validation score of 71.7% whereas for TFIDF vectorization, we got 72% 10 fold cross validation score.
* The performance of the model can be increased by using different classification models or neural networks besides changing the number of features for vectorization and also by using other pre-processing techniques like removing stop words etc.